In [ ]:
#fastai
!pip install -q fastbook==0.0.29

# #DuckDuckGo Search API
!pip install -q duckduckgo_search==5.3.0b4

ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data collection and cleaning

##Web scraping

In [ ]:
from fastbook import *
from fastai.vision.widgets import *
from tqdm.auto import tqdm
from duckduckgo_search import DDGS

#download from duckduckgo
def search_images_ddg(key, max_n=200):
     """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        (Adopted from https://github.com/deepanprabhu/duckduckgo-images-api)
     """
     url        = 'https://duckduckgo.com/'
     params     = {'q':key}
     res        = requests.post(url,data=params)
     searchObj  = re.search(r'vqd=([\d-]+)\&',res.text)
     if not searchObj: print('Token Parsing Failed !'); return
     requestUrl = url + 'i.js'
     headers    = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:71.0) Gecko/20100101 Firefox/71.0'}
     params     = (('l','us-en'),('o','json'),('q',key),('vqd',searchObj.group(1)),('f',',,,'),('p','1'),('v7exp','a'))
     urls       = []
     while True:
         try:
             res  = requests.get(requestUrl,headers=headers,params=params)
             data = json.loads(res.text)
             for obj in data['results']:
                 urls.append(obj['image'])
                 max_n = max_n - 1
                 if max_n < 1: return L(set(urls))     # dedupe
             if 'next' not in data: return L(set(urls))
             requestUrl = url + data['next']
         except:
             pass

def search_images_ddg_v2(key,max_n=200):
    """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        Note: Instead of using our own implementation, we using library instead!
    """
    responses = DDGS().images(
                          keywords=key,
                          region="wt-wt",
                          safesearch="off",
                          size=None,
                          color=None,
                          type_image=None,
                          layout=None,
                          license_image=None,
                          max_results=max_n,
                      )
    urls = [ r['image'] for r in responses ]
    return urls

In [ ]:
#save 
path = Path('images')

labels =  ['puncture wound']

#เอา label
MAX_N = 500

if not path.exists():
    path.mkdir()
for l in tqdm(labels):
    dest = (path/l)
    dest.mkdir(exist_ok=True)
    results = search_images_ddg_v2(l, max_n=MAX_N)
    download_images(dest, urls=results)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
#เช็คว่าได้รูปอะไรบ้าง
fns = get_image_files(path)
fns

In [ ]:
#ดูว่าหารูปไหนไม่เจอบ้าง
failed = verify_images(fns)
failed

(#3) [Path('images/puncture wound/cf2fe9c3-6a76-4127-b314-5484fec4a09d.jpg'),Path('images/puncture wound/32404100-a7c5-4c47-b249-2c2422907408.jpg'),Path('images/puncture wound/12a47094-08a3-4d43-8bab-6c83ec9da0b6.jpg')]

In [ ]:
#นำรูปที่หาไม่เจอออก
failed.map(Path.unlink)

(#7) [None,None,None,None,None,None,None]

In [ ]:
!zip -r /content/images.zip /content/images

##Data spiltting

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
import os
path = '/content/dataset'
print(os.listdir(path))

['bruises wound', 'burn wound', 'cut wound', 'laceration wound', 'abrasion wound', 'Stab_wound']


In [ ]:
splitfolders.ratio(path,seed=1337, output="splitted_dataset", ratio=(0.8, 0.1, 0.1))

Copying files: 870 files [00:00, 5353.78 files/s]


In [ ]:
!zip -r /content/splitted_dataset.zip /content/splitted_dataset

#Data augmentation

###Upsamling

https://www.alirookie.com/post/computer-vision-data-augmentation-code-sample

In [ ]:
!unzip /content/06train.zip -d data

In [ ]:
import os
import cv2
import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
from datetime import datetime
from imgaug.augmentables.batches import UnnormalizedBatch

In [ ]:
def count_files_in_folder(folder):
    files_count = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))])
    return files_count

def save_image(image, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    image_counter = count_files_in_folder(folder) + 1
    image_name = os.path.join(folder, f"{image_counter}.jpg")
    image.save(image_name)

def get_files_in_folder(folder):
    return [os.path.join(folder, name) for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))]

def list_oversample(initial_list, max_size):
    resized_array = []
    initial_length = len(initial_list)
    new_size = max_size - initial_length
    print('need to add', str(new_size))
    if new_size >= initial_length:
        augment_rate = int(new_size/initial_length)
        resized_array = initial_list * augment_rate
    else:
        resized_array = initial_list[:new_size]
    return resized_array

def save_image_array(image_array, folder):
    for image in image_array:
        # Convert RGBA images to RGB mode
        if image.mode == 'RGBA':
            image = image.convert('RGB')
        save_image(image, folder)

In [ ]:
# ia.seed(42)

# seq = iaa.Sequential([
#     iaa.Fliplr(0.5),  # horizontal flips
#     iaa.Crop(percent=(0, 0.1)),  # random crops
#     iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),  # Gaussian blur
#     iaa.LinearContrast((0.75, 1.5)),  # Contrast adjustment
#     iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5),  # Gaussian noise
#     iaa.Multiply((0.8, 1.2), per_channel=0.2),  # Brightness adjustment
#     iaa.Affine(
#         scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
#         translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
#         rotate=(-25, 25),
#         shear=(-8, 8)
#     )  # Affine transformations
# ], random_order=True)  # apply augmenters in random order

In [ ]:
import imgaug.augmenters as iaa

def sometimes(probability, aug):
  return iaa.Sometimes(probability, aug)

ia.seed(42)

crop_probability = 0.2
affine_probability = 0.7

seq = iaa.Sequential([
  iaa.Fliplr(0.5),
  iaa.Flipud(0.5),
  iaa.Crop(percent=(0, 0.2)),

  sometimes(crop_probability, iaa.CropToAspectRatio(9/16)),
  sometimes(crop_probability, iaa.CropToAspectRatio(3/4)),
  sometimes(crop_probability, iaa.CropToAspectRatio(2/3)),

  sometimes(affine_probability, iaa.Affine(
    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
    translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
    rotate=(-45, 45),
    shear=(-16, 16),
    order=[0, 1],
    cval=(0, 255),
    mode=ia.ALL
  )),
  iaa.Sharpen(alpha=(0, 0.3), lightness=(0.75, 1.5)),
  iaa.LinearContrast((0.75, 1.5)),
  iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5),
  iaa.Multiply((0.8, 1.2), per_channel=0.2),
], random_order=True)


In [ ]:
IMAGE_FOLDER = '/content/data/train'
image_subfolders = [os.path.join(IMAGE_FOLDER, subfolder) for subfolder in os.listdir(IMAGE_FOLDER)]
max_image_count = max([count_files_in_folder(subfolder) for subfolder in image_subfolders])
image_target_subfolders = [subfolder for subfolder in image_subfolders if count_files_in_folder(subfolder) < max_image_count]

In [ ]:
b4_count = [count_files_in_folder(subfolder) for subfolder in image_subfolders]
b4_count

[122, 81, 293, 48, 70, 80]

In [ ]:
for subfolder in image_target_subfolders:
    print(subfolder, 'with', str(count_files_in_folder(subfolder)), 'pic')
    start_time = datetime.now()
    image_files = get_files_in_folder(subfolder)
    synthetic_image_files = list_oversample(image_files, max_image_count)
    images = [np.array(Image.open(image_file)) for image_file in synthetic_image_files]
    augmented_images = seq(images=images)
    augmented_images = [Image.fromarray(image) for image in augmented_images]
    save_image_array(augmented_images, subfolder)
    print('at the end there is', str(count_files_in_folder(subfolder)))
    end_time = datetime.now()
    time_spent = end_time - start_time
    spent_minutes, spent_seconds = divmod(time_spent.days * 86400 + time_spent.seconds, 60)
    print("{} min {} sec".format(spent_minutes, spent_seconds))


/content/data/train/bruises wound with 122 pic
need to add 171
at the end there is 244
0 min 8 sec
/content/data/train/abrasion wound with 81 pic
need to add 212
at the end there is 243
0 min 8 sec
/content/data/train/Stab_wound with 48 pic
need to add 245
at the end there is 288
0 min 17 sec
/content/data/train/cut wound with 70 pic
need to add 223
at the end there is 280
0 min 10 sec
/content/data/train/laceration wound with 80 pic
need to add 213
at the end there is 240
0 min 8 sec


In [ ]:
[count_files_in_folder(subfolder) for subfolder in image_subfolders]

[244, 243, 293, 288, 280, 240]

In [ ]:
!zip -r /content/train.zip /content/data/train